In [1]:
from tqdm.notebook import tqdm
import math
import gym
import torch
import torch.optim as optim 
from torch.utils.tensorboard import SummaryWriter
from collections import deque

from active_rl.networks.dqn_atari import DQN, ENS_DQN
from active_rl.utils.memory import ReplayMemory, LabelledReplayMemory
from active_rl.utils.optimization import standard_optimization, AMN_optimization_ensemble
from active_rl.environments.atari_wrappers import make_atari, wrap_deepmind
from active_rl.utils.atari_utils import fp, ActionSelector, evaluate
from active_rl.utils.training_loop import collect_trajectories
from active_rl.utils.acquisition_functions import ens_BALD

In [2]:
env_name = 'Breakout'
env_raw = make_atari('{}NoFrameskip-v4'.format(env_name))
env = wrap_deepmind(env_raw, frame_stack=False, episode_life=True, clip_rewards=True)
c,h,w = c,h,w = fp(env.reset()).shape
n_actions = env.action_space.n

In [3]:
BATCH_SIZE = 64
LR = 0.0000625
GAMMA = 0.99
EPS_START = 1.
EPS_END = 0.05
EPS_DECAY = 1000000 
NUM_STEPS = 20000000
POLICY_UPDATE = 4
TARGET_UPDATE = 4000
EVALUATE_FREQ = 10000
MEMORY_CAPACITY = 100000

LABELLED_MEMORY_CAPACITY = 10000
UNLABELLED_MEMORY_CAPACITY = 10000
BATCH_LABEL_PERCENTAGE=0.1
TRAINING_ITERATIONS= int(2 * LABELLED_MEMORY_CAPACITY /  BATCH_SIZE)
TRAIN_AMN_FREQ = 1000

NAME = f'standard_rl_AMN_ens_exploration_{env_name}'

In [4]:
device = 'cuda:0'
policy_net = DQN(n_actions).to(device)
target_net = DQN(n_actions).to(device)
policy_net.apply(policy_net.init_weights)
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()
AMN_net = ENS_DQN(n_actions).to(device)
optimizer_value = optim.Adam(policy_net.parameters(), lr=LR, eps=1.5e-4)
optimizer_AMN = optim.Adam(AMN_net.parameters(), lr=LR, eps=1.5e-4)

In [5]:
memory = ReplayMemory(MEMORY_CAPACITY, [5,h,w], n_actions, device)
AMN_memory = LabelledReplayMemory(UNLABELLED_MEMORY_CAPACITY, LABELLED_MEMORY_CAPACITY, [5,h,w], n_actions, ens_BALD, AMN_net, device=device)
action_selector_explore = ActionSelector(EPS_START, EPS_END, policy_net, EPS_DECAY, n_actions, device)
action_selector_AMN = ActionSelector(EPS_START, EPS_END, policy_net, EPS_DECAY, n_actions, device)

In [6]:
steps_done = 0
writer = SummaryWriter(f'runs/{NAME}')

In [7]:
q = deque(maxlen=5)
done=True
eps = 0
episode_len = 0

In [ ]:
progressive = tqdm(range(NUM_STEPS), total=NUM_STEPS, ncols=400, leave=False, unit='b')
for step in progressive:
  if done:
    env.reset()
    sum_reward = 0
    episode_len = 0
    img, _, _, _ = env.step(1) # BREAKOUT specific !!!
    for i in range(10): # no-op
      n_frame, _, _, _ = env.step(0)
      n_frame = fp(n_frame)
      q.append(n_frame)
        
  # Select and perform an action
  state = torch.cat(list(q))[1:].unsqueeze(0)
  action, eps = action_selector_explore.select_action(state)
  n_frame, reward, done, info = env.step(action)
  n_frame = fp(n_frame)

  # 5 frame as memory
  q.append(n_frame)
  memory.push(torch.cat(list(q)).unsqueeze(0), action, reward, done) # here the n_frame means next frame from the previous time step
  episode_len += 1

  # Perform one step of the optimization (on the target network)
  if step % POLICY_UPDATE == 0 and step > 0:
    loss = standard_optimization(policy_net, target_net, optimizer_value, memory, batch_size=BATCH_SIZE, device=device)
    writer.add_scalar('training/loss_value_net', loss, step)
    
  # Update the target network, copying all weights and biases in DQN
  if step % TARGET_UPDATE == 0 and step > 0:
    target_net.load_state_dict(policy_net.state_dict())
    
  if step % TRAIN_AMN_FREQ == 0 and step > 0:
    collect_trajectories(UNLABELLED_MEMORY_CAPACITY, env, action_selector_AMN, AMN_memory)
    AMN_memory.label_sample(percentage=BATCH_LABEL_PERCENTAGE, batch_size=BATCH_SIZE)
    loss = 0
    for _ in range(TRAINING_ITERATIONS):
      loss += AMN_optimization_ensemble(AMN_net, target_net, optimizer_AMN, AMN_memory, batch_size=BATCH_SIZE, device=device)
    loss /= TRAINING_ITERATIONS
    writer.add_scalar('training/loss_AMN_net', loss, step)
    
  if step % EVALUATE_FREQ == 0 and step > 0:
    evaluated_reward = evaluate(step, AMN_net, device, env_raw, n_actions, eps=0.05, num_episode=15)
    writer.add_scalar('Performance/reward_vs_step', evaluated_reward, step)